<a href="https://colab.research.google.com/github/Jonathanpro/myaiblog/blob/master/_notebooks/2021-07-20-Topic_Modelling_with_Bert_Topic_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP - Topic Modelling with Bertopic - Training
> A tutorial for training a topic modelling classifier with Huggingface & Bertopic.

- toc: true 
- badges: true
- comments: false
- categories: [jupyter, NLP, TPU]
- image: images/chart-preview.png

# Enabling the GPU

Start this notebook with a GPU attached:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

# **Requirements**
After installing bertopic click on restart runtime to proceed

In [ ]:
!pip install bertopic

     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 15.7MB 190kB/s 
     |████████████████████████████████| 13.2MB 156kB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
     |████████████████████████████████| 6.4MB 34.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 92kB 13.7MB/s 
     |████████████████████████████████| 1.1MB 39.2MB/s 
     |████████████████████████████████| 2.5MB 37.9MB/s 
     |████████████████████████████████| 1.2MB 32.7MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 3.3MB 32.6MB/s 
     |████████████████████████████████| 901kB 47.9MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311711 sha256=4ef1b18772cfa55fd8938dac90f240d3a5affa40430b1b9ed5d1e74964fe5e8b
  Stored in directory: /root/.cache/pip/wh

In [ ]:
!pip install numba --upgrade

     |████████████████████████████████| 3.4MB 37.4MB/s 
     |████████████████████████████████| 25.3MB 120kB/s 
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

# **Load data**
You need to create a kaggle.json in your kaggle account and upload it via the below cell

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 62 bytes


In [ ]:
import kaggle

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

In [ ]:
api.dataset_download_file('pqbsbk/german-news-dataset',
                          file_name='data.csv')

True

In [ ]:
!unzip data.csv.zip

Archive:  data.csv.zip
  inflating: data.csv                


In [ ]:
!ls

data.csv  data.csv.zip	sample_data


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
df = df[df["text"].notnull()]

In [ ]:
df.count()

title        167280
text         167280
author       148280
published    167256
tags         167280
source       167280
dtype: int64

In [ ]:
df.head()

,title,text,author,published,tags,source
0,Warum nicht mal die Russen Putins Impfstoff wo...,Im Kreml sollte es in diesen Tagen eigentlich ...,Pavel Lokshin,1602234820,"Ausland, „Sputnik V“",welt
2,Welternährungsprogramm erhält den Friedensnobe...,Der Preis setze ein Zeichen für die Millionen ...,"'Hannes Stein', 'Birgit Herden', 'Dirk Schümer...",1602234160,"Ausland, Oslo",welt
4,In welchen deutschen Großstädten die Lage krit...,Die Kanzlerin berät mit den Bürgermeistern der...,"'Daniele Raffaele Gambone', 'Christoph B. Schi...",1602233883,"Deutschland, Corona-Pandemie",welt
7,"Klopp beschreibt den Moment, als er vom Liverp...","Anzeige\n\nWie die Zeit vergeht, merkt auch er...","'Lars Gartenschläger', 'Lutz Wöckener', 'Karl-...",1602233011,"Fußball, „Werde ich nie vergessen“",welt
8,Das beste FIFA aller Zeiten – mit einem großen...,Anzeige\n\nDie Corona-Krise hat auch vor dem e...,Marcel Reich,1602232402,"Wirtschaft, FIFA 21 im Test",welt


In [ ]:
"""
import torch_xla
import torch_xla.core.xla_model as xm

device = xm.xla_device()
"""

'\nimport torch_xla\nimport torch_xla.core.xla_model as xm\n\ndevice = xm.xla_device()\n'

# **Train model**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic

sentence_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda")
# sentence_model = sentence_model.to(device)
topic_model = BERTopic(low_memory = True, embedding_model=sentence_model, calculate_probabilities=True, verbose=True)


In [ ]:
# topics, probs = topic_model.fit_transform(list(df["text"]))
topic_model.fit(list(df["text"])[:25000])

2021-07-19 14:26:53,623 - BERTopic - Transformed documents to Embeddings


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2021-07-19 14:27:41,815 - BERTopic - Reduced dimensionality with UMAP
2021-07-19 14:32:21,916 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()


# dd/mm/YY H:M:S
dt_string = now.strftime("%Y_%m_%d__%H_%M_%S")
print("date and time =", dt_string)	



date and time = 2021_07_19__14_33_09


In [ ]:
topic_model.save("/content/gdrive/MyDrive/tm_bert_topic_{}".format(dt_string), save_embedding_model=True)	

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
??topic_model.save

In [ ]:
freq = topic_model.get_topic_info()
freq.head(20)

,Topic,Count,Name
0,-1,8667,-1_sagt_quelle_jahr_denn
1,0,264,0_arbeitnehmer_arbeitgeber_arbeiten_job
2,1,248,1_migranten_flüchtlinge_flüchtlingslager_griec...
3,2,205,2_pizza_essen_ernährung_burger
4,3,191,3_bahn_stuttgart_hamburg_scheuer
5,4,180,4_rki_neuinfektionen_starben_coronavirus
6,5,178,5_kandidaten_stuttgart_stuttgarter_kienzle
7,6,173,6_rechtsextremismus_rechtsextreme_rechtsextrem...
8,7,169,7_impfstoff_impfstoffe_impfstoffs_impfungen
9,8,168,8_demonstration_demonstranten_polizei_berlin
